# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [1]:
# Imports

import pandas as pd
import time

In [2]:
# Importing from .py files

from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset
from models import create_model

2025-03-07 17:30:47.036727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741368647.055656  257261 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741368647.061326  257261 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 17:30:47.081622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [3]:
train_list = load_data(TRAIN_TXT)
test_list = load_data(TEST_TXT)

# Create dictionaries to store datasets for different batch sizes
train_datasets = {}
test_datasets = {}

start_time = time.time()
# Iterate over batch sizes and create datasets
for batch_size in BATCH_SIZES:
    train_datasets[batch_size] = create_dataset(
        train_list, batch_size, is_training=True
    )
    test_datasets[batch_size] = create_dataset(test_list, batch_size, is_training=False)
print(f"Time taken to create datasets: {time.time() - start_time} seconds")

I0000 00:00:1741389566.814773    2172 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22455 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:07:00.0, compute capability: 8.6


In [ ]:
# Run model experiments
exp_name = "model-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(exp, exp_name)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model,
        base_model,
        exp_name,
        exp,
        train_dataset,
        test_dataset,
        train_list,
        test_list,
    )

Defining model: resnet50 no-pretraining no-warmup
In training loop: resnet50 no-pretraining no-warmup
Recompiling model at epoch 0 (Optimizer changed)


I0000 00:00:1741389596.130585    2172 service.cc:148] XLA service 0x16dcf2b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741389596.130751    2172 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-03-07 23:19:56.939286: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741389600.328814    2172 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-07 23:20:08.091401: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6302_0', 112 bytes spill stores, 224 bytes spill loads

2025-03-07 23:20:10.588146: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11157', 24 byte

Time taken for training one epoch: 112.77s
Epoch 0 training loss: 0.17, acc: 0.89, f1: 0.56, mAP: 0.74
Time taken for testing one epoch: 46.13s
Epoch 0 test loss: 0.26, acc: 0.74, f1: 0.04, mAP: 0.41
Time taken for training one epoch: 51.63s
Epoch 1 training loss: 0.09, acc: 0.97, f1: 0.72, mAP: 0.87
Time taken for testing one epoch: 36.58s
Epoch 1 test loss: 0.21, acc: 0.87, f1: 0.41, mAP: 0.63
Time taken for training one epoch: 52.76s
Epoch 2 training loss: 0.07, acc: 0.98, f1: 0.81, mAP: 0.93
Time taken for testing one epoch: 37.75s
Epoch 2 test loss: 0.27, acc: 0.86, f1: 0.60, mAP: 0.73
Time taken for training one epoch: 53.92s
Epoch 3 training loss: 0.05, acc: 0.99, f1: 0.86, mAP: 0.96
Time taken for testing one epoch: 38.22s
Epoch 3 test loss: 0.17, acc: 0.91, f1: 0.65, mAP: 0.79
Time taken for training one epoch: 52.90s
Epoch 4 training loss: 0.04, acc: 0.99, f1: 0.89, mAP: 0.97
Time taken for testing one epoch: 37.72s
Epoch 4 test loss: 0.17, acc: 0.91, f1: 0.66, mAP: 0.80
Time

In [ ]:
# Determine the best experiment of the 9 model experiments

df = pd.read_csv(RESULTS_DIR / f"model-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_model_experiment_config = next(
    exp for exp in experiments["model-experiments"] if exp.id == best_id
)

best_model_experiment_config

In [ ]:
# Run hyperparameter experiments

exp_name = "hyperparameter-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(exp, exp_name, best_model_experiment_config)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model,
        base_model,
        exp_name,
        exp,
        train_dataset,
        test_dataset,
        train_list,
        test_list,
    )

Defining model: batch_size: 64, learning_rate: 0.001
Reusing net_name: ['resnet50', 'ResNet50'], train_from_scratch: False, warm_up: True from best model experiment


In training loop: batch_size: 64, learning_rate: 0.001


I0000 00:00:1741335488.804120 1486723 service.cc:148] XLA service 0x2a477e30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741335488.804155 1486723 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2025-03-07 08:18:09.295413: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741335491.711692 1486723 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-07 08:18:14.921285: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6300_0', 204 bytes spill stores, 204 bytes spill loads

2025-03-07 08:18:16.144058: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6300', 192 b

Time taken for training one epoch: 84.77s
Epoch 0 training loss: 0.17, acc: 0.89, f1: 0.57, mAP: 0.74


2025-03-07 08:19:21.545454: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1738_0', 112 bytes spill stores, 224 bytes spill loads

2025-03-07 08:19:21.619983: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1738', 192 bytes spill stores, 512 bytes spill loads

2025-03-07 08:19:21.768963: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1745', 220 bytes spill stores, 220 bytes spill loads



Time taken for testing one epoch: 21.74s
Epoch 0 test loss: 0.27, acc: 0.58, f1: 0.00, mAP: 0.20
Time taken for training one epoch: 68.98s
Epoch 1 training loss: 0.08, acc: 0.98, f1: 0.78, mAP: 0.91
Time taken for testing one epoch: 18.13s
Epoch 1 test loss: 0.25, acc: 0.82, f1: 0.12, mAP: 0.54
Unfreezing base model at epoch 2
Time taken for training one epoch: 68.72s
Epoch 2 training loss: 0.05, acc: 0.99, f1: 0.87, mAP: 0.96
Time taken for testing one epoch: 18.50s
Epoch 2 test loss: 0.30, acc: 0.82, f1: 0.35, mAP: 0.62
Time taken for training one epoch: 68.95s
Epoch 3 training loss: 0.04, acc: 0.99, f1: 0.90, mAP: 0.98
Time taken for testing one epoch: 18.42s
Epoch 3 test loss: 0.17, acc: 0.91, f1: 0.67, mAP: 0.81
Time taken for training one epoch: 67.92s
Epoch 4 training loss: 0.03, acc: 1.00, f1: 0.92, mAP: 0.98
Time taken for testing one epoch: 18.50s
Epoch 4 test loss: 0.25, acc: 0.86, f1: 0.56, mAP: 0.72
Time taken for training one epoch: 69.07s
Epoch 5 training loss: 0.02, acc

In [4]:
# Determine the best experiment of the 9 hyperparameter experiments

df = pd.read_csv(RESULTS_DIR / f"hyperparameter-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp.id == best_id
)

best_hyperparameter_experiment_config

ExperimentConfig(id=15, title='batch_size: 64, learning_rate: 0.001', net_name=['resnet50', 'ResNet50'], train_from_scratch=False, warm_up=True, batch_size=64, n_epochs=12, last_layer_activation='sigmoid', learning_rate=0.001, loss='binary_crossentropy', classifier_head='default')

In [ ]:
# Run augmentation experiments

exp_name = "augmentation-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(
        exp, exp_name, best_hyperparameter_experiment_config
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model,
        exp_name,
        exp,
        train_dataset,
        test_dataset,
        train_list,
        test_list,
    )

In [6]:
# Run classifier head experiments

exp_name = "classfier_head-experiments"
for exp in experiments[exp_name][1:]:

    # Create the model
    base_model, model = create_model(
        exp, exp_name, best_hyperparameter_experiment_config
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model,
        base_model,
        exp_name,
        exp,
        train_dataset,
        test_dataset,
        train_list,
        test_list,
    )

Defining model: classifier_head: attention
Reusing from best hyperparameter experiment:
	net_name: ['resnet50', 'ResNet50'],
	train_from_scratch: False,
	warm_up: True,
	batch_size: 64,
	learning_rate: 0.001,
	loss: binary_crossentropy,
	last_layer_activation: sigmoid
In training loop: classifier_head: attention
Freezing base model layers for warmup.
Recompiling model at epoch 0 (Optimizer changed)


I0000 00:00:1741364154.375972  177183 service.cc:148] XLA service 0xfca9af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741364154.376114  177183 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-03-07 16:15:54.725472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741364156.612851  177183 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-07 16:16:05.989336: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6880', 96 bytes spill stores, 96 bytes spill loads

2025-03-07 16:16:06.177585: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3157', 216 bytes spi

Time taken for training one epoch: 69.78s
Epoch 0 training loss: 0.26, acc: 0.74, f1: 0.38, mAP: 0.55


2025-03-07 16:17:04.877941: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1758_0', 112 bytes spill stores, 224 bytes spill loads

2025-03-07 16:17:06.722508: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1758', 192 bytes spill stores, 512 bytes spill loads



Time taken for testing one epoch: 45.69s
Epoch 0 test loss: 0.18, acc: 0.86, f1: 0.31, mAP: 0.65
Time taken for training one epoch: 36.24s
Epoch 1 training loss: 0.17, acc: 0.86, f1: 0.45, mAP: 0.67
Time taken for testing one epoch: 27.31s
Epoch 1 test loss: 0.17, acc: 0.88, f1: 0.37, mAP: 0.67
Time taken for training one epoch: 35.14s
Epoch 2 training loss: 0.16, acc: 0.88, f1: 0.48, mAP: 0.70
Time taken for testing one epoch: 26.71s
Epoch 2 test loss: 0.17, acc: 0.89, f1: 0.39, mAP: 0.68
Unfreezing base model at epoch 3
Recompiling model at epoch 3 (Optimizer changed)
Time taken for training one epoch: 116.15s
Epoch 3 training loss: 0.15, acc: 0.91, f1: 0.60, mAP: 0.76
Time taken for testing one epoch: 36.76s
Epoch 3 test loss: 0.34, acc: 0.61, f1: 0.00, mAP: 0.25
Time taken for training one epoch: 47.18s
Epoch 4 training loss: 0.09, acc: 0.97, f1: 0.74, mAP: 0.87
Time taken for testing one epoch: 27.36s
Epoch 4 test loss: 0.29, acc: 0.79, f1: 0.14, mAP: 0.48
Time taken for training 